# 01 Data Exploration & Preprocessing

Explore and preprocess the SMS Spam Collection Dataset with 5,574 messages.

**Dataset**: `hw3_dataset/sms_spam_no_header.csv`

## 1. Setup and Imports

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Import preprocessing functions
from src.preprocessing import (
    load_sms_data,
    clean_text,
    tokenize_and_stem,
    vectorize_text,
    preprocess_pipeline
)

# Set styling
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully!")

## 2. Load Dataset

In [ ]:
# Load the formal SMS Spam dataset
csv_path = '../hw3_dataset/sms_spam_no_header.csv'

texts, labels, label_names = load_sms_data(csv_path)

print(f"✅ Dataset loaded successfully from: {csv_path}")
print(f"\n📊 Dataset Statistics:")
print(f"   Total messages: {len(texts):,}")
print(f"   Label names: {label_names}")

ham_count = sum(1 for l in labels if l == 0)
spam_count = sum(1 for l in labels if l == 1)

print(f"\n📈 Label Distribution:")
print(f"   Ham (Legitimate): {ham_count:,} ({ham_count/len(labels)*100:.1f}%)")
print(f"   Spam: {spam_count:,} ({spam_count/len(labels)*100:.1f}%)")

## 3. Explore Sample Messages

In [ ]:
# Display sample HAM messages
print("Sample HAM messages (first 3):")
print("=" * 100)
ham_count_shown = 0
for text, label in zip(texts, labels):
    if label == 0 and ham_count_shown < 3:
        print(f"Length: {len(text)} chars")
        print(f"Text: {text}")
        print("-" * 100)
        ham_count_shown += 1

In [ ]:
# Display sample SPAM messages
print("Sample SPAM messages (first 3):")
print("=" * 100)
spam_count_shown = 0
for text, label in zip(texts, labels):
    if label == 1 and spam_count_shown < 3:
        print(f"Length: {len(text)} chars")
        print(f"Text: {text}")
        print("-" * 100)
        spam_count_shown += 1

## 4. Text Cleaning Demonstration

In [ ]:
# Show text cleaning process
sample_texts = [
    "FreeMsg: Hey check this out! Visit http://example.com NOW!!! PRIZE MONEY!!!",
    "Hello, how are you doing? Email me@example.com for details.",
    "U dun say so early hor... U c already then say... 123 456!!!"
]

print("Text Cleaning Pipeline Demonstration:")
print("=" * 100)

for i, text in enumerate(sample_texts, 1):
    cleaned = clean_text(text)
    print(f"\nExample {i}:")
    print(f"  Original:  {text}")
    print(f"  Cleaned:   {cleaned}")

## 5. Tokenization & Stemming

In [ ]:
# Demonstrate tokenization and stemming
example_texts = [
    "running runners running quickly run",
    "connected connecting connection"
]

print("Tokenization & Stemming Demonstration:")
print("=" * 100)

for text in example_texts:
    cleaned = clean_text(text)
    tokens = tokenize_and_stem(cleaned, remove_stopwords=False)
    
    print(f"\nOriginal:  {text}")
    print(f"Cleaned:   {cleaned}")
    print(f"Tokens:    {tokens}")

## 6. Message Length Distribution

In [ ]:
# Analyze message lengths
message_lengths = [len(text) for text in texts]

print(f"Message Length Statistics:")
print(f"  Min length: {min(message_lengths)} characters")
print(f"  Max length: {max(message_lengths)} characters")
print(f"  Mean length: {np.mean(message_lengths):.1f} characters")
print(f"  Median length: {np.median(message_lengths):.1f} characters")

# Plot distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# All messages
axes[0].hist(message_lengths, bins=50, color='skyblue', edgecolor='black')
axes[0].set_xlabel('Message Length (characters)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Message Lengths')
axes[0].grid(alpha=0.3)

# By label
ham_lengths = [len(text) for text, label in zip(texts, labels) if label == 0]
spam_lengths = [len(text) for text, label in zip(texts, labels) if label == 1]

axes[1].hist([ham_lengths, spam_lengths], bins=50, label=['Ham', 'Spam'], 
              color=['green', 'red'], alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Message Length (characters)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Message Length by Label')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Complete Preprocessing Pipeline with TF-IDF

In [ ]:
# Run complete preprocessing pipeline
print("Running complete preprocessing pipeline...")
print("This may take a few moments...\n")

result = preprocess_pipeline(csv_path, method='tfidf', max_features=5000, min_df=1, max_df=1.0)

X = result['X']
y = result['y']
feature_names = result['feature_names']
metadata = result['metadata']

print(f"✅ Preprocessing complete!")
print(f"\n📊 Vectorization Statistics:")
print(f"   Feature matrix shape: {X.shape}")
print(f"   Number of features: {len(feature_names)}")
print(f"   Number of non-zero entries: {X.nnz:,}")
print(f"   Sparsity: {metadata['sparsity']*100:.2f}%")
print(f"   Method: {metadata['method'].upper()}")

## 8. Feature Analysis

In [ ]:
# Analyze top features
tfidf_vectorizer = result['vectorizer']
feature_importance = np.asarray(X.mean(axis=0)).ravel()

# Get top features
top_indices = np.argsort(feature_importance)[-20:][::-1]
top_features = [feature_names[i] for i in top_indices]
top_scores = feature_importance[top_indices]

print("Top 20 Most Important Features (by average TF-IDF score):")
print("=" * 60)
for i, (feature, score) in enumerate(zip(top_features, top_scores), 1):
    print(f"{i:2d}. {feature:20s} Score: {score:.6f}")

## 9. Label Distribution Visualization

In [ ]:
# Create label distribution visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart
labels_pie = ['Ham', 'Spam']
sizes = [ham_count, spam_count]
colors = ['#2ca02c', '#d62728']
explode = (0.05, 0.05)

axes[0].pie(sizes, explode=explode, labels=labels_pie, autopct='%1.1f%%',
            colors=colors, startangle=90, textprops={'fontsize': 12, 'weight': 'bold'})
axes[0].set_title('Label Distribution', fontsize=14, fontweight='bold')

# Bar chart
axes[1].bar(labels_pie, sizes, color=colors, edgecolor='black', linewidth=2)
axes[1].set_ylabel('Count', fontsize=12, fontweight='bold')
axes[1].set_title('Message Count by Label', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

# Add count labels on bars
for i, (label, size) in enumerate(zip(labels_pie, sizes)):
    axes[1].text(i, size + 100, f'{size:,}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 10. Summary Statistics

In [ ]:
# Print comprehensive summary
print("\n" + "=" * 80)
print("DATA EXPLORATION SUMMARY")
print("=" * 80)

print("\n📊 DATASET INFORMATION")
print(f"  Dataset File: {csv_path}")
print(f"  Total Messages: {len(texts):,}")
print(f"  Ham Messages: {ham_count:,} ({ham_count/len(labels)*100:.1f}%)")
print(f"  Spam Messages: {spam_count:,} ({spam_count/len(labels)*100:.1f}%)")

print("\n📏 MESSAGE LENGTH STATISTICS")
print(f"  Min: {min(message_lengths)} characters")
print(f"  Max: {max(message_lengths)} characters")
print(f"  Mean: {np.mean(message_lengths):.1f} characters")
print(f"  Median: {np.median(message_lengths):.1f} characters")

print("\n🔤 PREPROCESSING RESULTS")
print(f"  Vectorization Method: TF-IDF")
print(f"  Total Features: {len(feature_names):,}")
print(f"  Feature Matrix Shape: {X.shape}")
print(f"  Non-zero Entries: {X.nnz:,}")
print(f"  Sparsity: {metadata['sparsity']*100:.2f}%")

print("\n✅ Data exploration complete!")
print("=" * 80)

# SMS Spam Data Exploration and Preprocessing Demo

This notebook demonstrates the complete data preprocessing pipeline for SMS spam classification:
1. Loading SMS spam data from CSV
2. Data exploration and statistics
3. Text cleaning and normalization
4. Tokenization and stemming
5. Bag-of-Words (BoW) vectorization
6. TF-IDF vectorization
7. Feature analysis and visualization

## 1. Import Required Libraries

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Import preprocessing functions
from src.preprocessing import (
    load_sms_data,
    clean_text,
    tokenize_and_stem,
    vectorize_text,
    preprocess_pipeline
)

print("✅ Libraries imported successfully!")

## 2. Load and Explore Data

In [ ]:
# Load sample data
csv_path = '../data/sample_sms_spam.csv'

texts, labels, label_names = load_sms_data(csv_path)

print(f"Dataset loaded successfully!")
print(f"Total messages: {len(texts)}")
print(f"Label names: {label_names}")
print(f"\nLabel distribution:")
print(f"  Ham: {sum(1 for l in labels if l == 0)}")
print(f"  Spam: {sum(1 for l in labels if l == 1)}")

## 3. Display Sample Messages

In [ ]:
# Show sample ham messages
print("Sample HAM messages (first 2):")
print("=" * 80)
for i, (text, label) in enumerate(zip(texts[:5], labels[:5])):
    if label == 0:
        print(f"Message {i+1}:")
        print(f"  Length: {len(text)} characters")
        print(f"  Text: {text[:100]}...\n")

print("\nSample SPAM messages (first 2):")
print("=" * 80)
for i, (text, label) in enumerate(zip(texts, labels)):
    if label == 1:
        print(f"Message {i+1}:")
        print(f"  Length: {len(text)} characters")
        print(f"  Text: {text[:100]}...\n")

## 4. Text Cleaning Demonstration

In [ ]:
# Show cleaning process
sample_texts = [
    "FreeMsg: Hey check this out! Visit http://example.com NOW!!!",
    "Hello, how are you? Email me@example.com",
    "U dun say so early hor... 123 456"
]

print("Text Cleaning Demonstration:")
print("=" * 80)
for text in sample_texts:
    cleaned = clean_text(text)
    print(f"Original: {text}")
    print(f"Cleaned:  {cleaned}")
    print()

## 5. Tokenization and Stemming

In [ ]:
# Show tokenization and stemming
sample_cleaned = "running runner runs quickly fast"

tokens = tokenize_and_stem(sample_cleaned, remove_stopwords=False)
tokens_no_stopwords = tokenize_and_stem(sample_cleaned, remove_stopwords=True)

print("Tokenization and Stemming:")
print(f"Original text: {sample_cleaned}")
print(f"\nTokens (with stopwords): {tokens}")
print(f"Tokens (without stopwords): {tokens_no_stopwords}")

## 6. Complete Preprocessing Pipeline

In [ ]:
# Run full preprocessing pipeline
result = preprocess_pipeline(csv_path, method='tfidf', max_features=1000)

X = result['X']
y = result['y']
feature_names = result['feature_names']
metadata = result['metadata']

print("Preprocessing Pipeline Results:")
print("=" * 80)
print(f"Feature matrix shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Number of features: {len(feature_names)}")
print(f"\nMetadata:")
for key, value in metadata.items():
    if key == 'label_distribution':
        print(f"  {key}: {value}")
    else:
        print(f"  {key}: {value}")

## 7. Feature Analysis

In [ ]:
# Show most important features
feature_importance = np.asarray(X.mean(axis=0)).ravel()
top_feature_indices = np.argsort(feature_importance)[-20:]

print("Top 20 Most Important Features (by TF-IDF):")
print("=" * 80)
for idx in reversed(top_feature_indices):
    print(f"{feature_names[idx]:20s} - Score: {feature_importance[idx]:.4f}")

## 8. Visualization: Label Distribution

In [ ]:
# Visualize label distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Bar plot
labels_count = [sum(y == 0), sum(y == 1)]
axes[0].bar(['Ham', 'Spam'], labels_count, color=['green', 'red'], alpha=0.7)
axes[0].set_title('Label Distribution')
axes[0].set_ylabel('Count')
for i, v in enumerate(labels_count):
    axes[0].text(i, v + 0.1, str(v), ha='center')

# Pie chart
axes[1].pie(labels_count, labels=['Ham', 'Spam'], autopct='%1.1f%%',
            colors=['green', 'red'], startangle=90)
axes[1].set_title('Label Proportion')

plt.tight_layout()
plt.show()

print(f"Ham: {labels_count[0]} ({100*labels_count[0]/len(y):.1f}%)")
print(f"Spam: {labels_count[1]} ({100*labels_count[1]/len(y):.1f}%)")

## 9. Bag-of-Words Comparison

In [ ]:
# Run preprocessing with Bag-of-Words
result_bow = preprocess_pipeline(csv_path, method='bow', max_features=1000)

X_bow = result_bow['X']
feature_names_bow = result_bow['feature_names']

print("Vectorization Method Comparison:")
print("=" * 80)
print(f"TF-IDF:")
print(f"  Matrix shape: {X.shape}")
print(f"  Data type: {X.dtype}")
print(f"  Sparsity: {(1 - X.nnz / (X.shape[0] * X.shape[1])):.2%}")
print(f"\nBag-of-Words:")
print(f"  Matrix shape: {X_bow.shape}")
print(f"  Data type: {X_bow.dtype}")
print(f"  Sparsity: {(1 - X_bow.nnz / (X_bow.shape[0] * X_bow.shape[1])):.2%}")

## 10. Summary and Next Steps

In [ ]:
# Load the formal SMS Spam dataset
csv_path = '../hw3_dataset/sms_spam_no_header.csv'

texts, labels, label_names = load_sms_data(csv_path)

print(f"✅ Dataset loaded successfully from: {csv_path}")
print(f"📊 Dataset Statistics:")
print(f"   Total messages: {len(texts):,}")
print(f"   Label names: {label_names}")
print(f"\n📈 Label distribution:")
ham_count = sum(1 for l in labels if l == 0)
spam_count = sum(1 for l in labels if l == 1)
print(f"   Ham (Legitimate): {ham_count:,} ({ham_count/len(labels)*100:.1f}%)")
print(f"   Spam: {spam_count:,} ({spam_count/len(labels)*100:.1f}%)")